In [2]:
import os
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [4]:
os.chdir('/content/drive/Shareddrives/GOV.UK teams/2020-2021/Data labs/govNER_ContentMetadata')

In [5]:
pd.read_csv('./Data/IntermediateData/test.csv').head()

,Unnamed: 0,text,text_token,labels,updated,original_labels,base_path,sampled,label_list,original_file,new_label_list_id
0,33165,"PDF , 1.49MB , 142 pages This file may not be ...","['PDF', ',', '1.49MB', ',', '142', 'pages', 'T...","[[59, 64, 'PERSON']]",NaN,NaN,/government/publications/guidance-for-driving-...,True,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",line_by_line_NER_data_sampled_12062020_more_en...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0,..."
1,87484,The minimum standards of training for healthca...,"['The', 'minimum', 'standards', 'of', 'trainin...","[[25, 33, 'STATE'], [49, 64, 'PERSON'], [132, ...",NaN,NaN,/government/publications/immunisation-training...,True,"['O', 'O', 'O', 'O', 'STATE', 'O', 'O', 'PERSO...",line_by_line_NER_data_sampled_12062020_more_en...,"[0, 0, 0, 0, 12, 0, 0, 10, 10, 0, 0, 0, 0, 0, ..."
2,200948,Imran Gulamhuseinwala has spent over 20 years ...,"['Imran', 'Gulamhuseinwala', 'has', 'spent', '...","[[37, 45, 'DATE'], [125, 135, 'ORGANIZATION']]",NaN,NaN,/government/news/imran-gulamhuseinwala-obe-and...,True,"['O', 'O', 'O', 'O', 'O', 'DATE', 'DATE', 'O',...",line_by_line_NER_data_sampled_09062020_more_en...,"[0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,241585,"In a report released Friday , in London and Ad...","['In', 'a', 'report', 'released', 'Friday', ',...","[[5, 11, 'CONTACT'], [21, 27, 'DATE'], [33, 39...",NaN,NaN,Sentence: 28104,False,"['O', 'O', 'CONTACT', 'O', 'DATE', 'O', 'O', '...",line_by_line_NER_data_sampled_09062020_more_en...,"[0, 0, 1, 0, 2, 0, 0, 6, 0, 0, 0, 0, 6, 0, 0, ..."
4,80747,Find out how to contact the Adjudicator 's Off...,"['Find', 'out', 'how', 'to', 'contact', 'the',...","[[16, 23, 'CONTACT'], [28, 39, 'PERSON']]",NaN,NaN,/guidance/contact-the-adjudicators-office,True,"['O', 'O', 'O', 'O', 'CONTACT', 'O', 'PERSON',...",line_by_line_NER_data_sampled_12062020_more_en...,"[0, 0, 0, 0, 1, 0, 10, 0, 0, 0]"


In [6]:
! pip install datasets transformers seqeval


     |████████████████████████████████| 298 kB 5.3 MB/s 
     |████████████████████████████████| 3.1 MB 36.3 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 59 kB 5.5 MB/s 
     |████████████████████████████████| 243 kB 51.1 MB/s 
     |████████████████████████████████| 1.1 MB 31.2 MB/s 
     |████████████████████████████████| 132 kB 50.0 MB/s 
     |████████████████████████████████| 596 kB 40.5 MB/s 
     |████████████████████████████████| 3.3 MB 35.2 MB/s 
     |████████████████████████████████| 895 kB 36.0 MB/s 
     |████████████████████████████████| 271 kB 48.6 MB/s 
     |████████████████████████████████| 160 kB 52.7 MB/s 
     |████████████████████████████████| 192 kB 40.9 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=f5efd104ba1f4732e3df3b10f9547217bf756cad681eb3cf84cf9a6caa3d73ce
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72

In [7]:
import transformers

print(transformers.__version__)

4.12.5


In [12]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
dataset_name = "conll2003"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [9]:
from datasets import load_dataset, load_metric

In [9]:
datasets = load_dataset(dataset_name)

Reusing dataset conll2003 (/root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6)


  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [11]:
datasets["train"][0]

{'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'id': '0',
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.']}

In [12]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], names_file=None, id=None), length=-1, id=None)

In [13]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [14]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [15]:
show_random_elements(datasets["train"])

,id,tokens,pos_tags,chunk_tags,ner_tags
0,1591,"[Gupta, said, there, might, be, an, increase, in, the, number, of, people, infiltrating, the, Kashmir, valley, to, create, disturbance, in, the, region, .]","[NNP, VBD, EX, MD, VB, DT, NN, IN, DT, NN, IN, NNS, VBG, DT, NNP, NN, TO, VB, NN, IN, DT, NN, .]","[B-NP, B-VP, B-NP, B-VP, I-VP, B-NP, I-NP, B-PP, B-NP, I-NP, B-PP, B-NP, B-VP, B-NP, I-NP, I-NP, B-VP, I-VP, B-NP, B-PP, B-NP, I-NP, O]","[B-PER, O, O, O, O, O, O, O, O, O, O, O, O, O, B-LOC, O, O, O, O, O, O, O, O]"
1,6594,"[Mauritania, v, Benin, postponed, to, Friday]","[NNP, FW, NNP, VBD, TO, NNP]","[B-NP, I-NP, B-NP, B-VP, B-PP, B-NP]","[B-LOC, O, B-LOC, O, O, O]"
2,12470,"[Sandra, Dopfer, (, Austria, ), beat, Nanne, Dahlman, (, Finland, ), 2-6, 6-2]","[NNP, NNP, (, NNP, ), VB, NNP, NNP, (, NNP, ), CD, CD]","[B-NP, I-NP, O, B-NP, O, B-VP, B-NP, I-NP, O, B-NP, O, B-NP, I-NP]","[B-PER, I-PER, O, B-LOC, O, O, B-PER, I-PER, O, B-LOC, O, O, O]"
3,1566,"[Britain, says, death, of, its, citizen, will, sour, ties, .]","[NNP, VBZ, NN, IN, PRP$, NN, MD, VB, NNS, .]","[B-NP, B-VP, B-NP, B-PP, B-NP, I-NP, B-VP, I-VP, B-NP, O]","[B-LOC, O, O, O, O, O, O, O, O, O]"
4,10567,"[The, 24-year-old, Gabrich, ,, who, signed, for, an, undisclosed, fee, from, the, Argentine, side, Newell, Old, Boys, ,, is, set, to, join, Dutch, international, Patrick, Kluivert, in, the, Ajax, forward, line, .]","[DT, JJ, NNP, ,, WP, VBD, IN, DT, JJ, NN, IN, DT, JJ, NN, NNP, NNP, NNPS, ,, VBZ, VBN, TO, VB, JJ, JJ, NNP, NNP, IN, DT, NNP, JJ, NN, .]","[B-NP, I-NP, I-NP, O, B-NP, B-VP, B-PP, B-NP, I-NP, I-NP, B-PP, B-NP, I-NP, I-NP, I-NP, I-NP, I-NP, O, B-VP, I-VP, I-VP, I-VP, B-NP, I-NP, I-NP, I-NP, B-PP, B-NP, I-NP, I-NP, I-NP, O]","[O, O, B-PER, O, O, O, O, O, O, O, O, O, B-MISC, O, B-ORG, I-ORG, I-ORG, O, O, O, O, O, B-MISC, O, B-PER, I-PER, O, O, B-ORG, O, O, O]"
5,13732,"[A, consortium, led, by, Thailand, 's, Siam, Commercial, Bank, Plc, has, secured, at, auction, the, right, to, sell, two, state, agency, bond, issues, worth, a, combined, 3.73, billion, baht, ,, an, official, at, the, bank, said, on, Thursday, .]","[DT, NN, VBN, IN, NNP, POS, NNP, NNP, NNP, NNP, VBZ, VBN, IN, NN, DT, NN, TO, VB, CD, NN, NN, NN, NNS, JJ, DT, VBN, CD, CD, NN, ,, DT, NN, IN, DT, NN, VBD, IN, NNP, .]","[B-NP, I-NP, B-VP, B-PP, B-NP, B-NP, I-NP, I-NP, I-NP, I-NP, B-VP, I-VP, B-PP, B-NP, B-NP, I-NP, B-VP, I-VP, B-NP, I-NP, I-NP, I-NP, I-NP, B-ADJP, B-NP, I-NP, I-NP, I-NP, I-NP, O, B-NP, I-NP, B-PP, B-NP, I-NP, B-VP, B-PP, B-NP, O]","[O, O, O, O, B-LOC, O, B-ORG, I-ORG, I-ORG, I-ORG, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
6,10070,"[6., Jans, Koerts, (, Netherlands, ), Palmans]","[CD, NNPS, NNP, (, NNP, ), NNS]","[B-NP, I-NP, I-NP, I-NP, I-NP, O, B-NP]","[O, B-PER, I-PER, O, B-LOC, O, B-ORG]"
7,179,"[Kabul, is, controlled, by, President, Burhanuddin, Rabbani, 's, government, ,, which, Taleban, is, fighting, to, overthrow, .]","[NNP, VBZ, VBN, IN, NNP, NNP, NNP, POS, NN, ,, WDT, NNP, VBZ, VBG, TO, VB, .]","[B-NP, B-VP, I-VP, B-PP, B-NP, I-NP, I-NP, B-NP, I-NP, O, B-NP, B-NP, B-VP, I-VP, I-VP, I-VP, O]","[B-LOC, O, O, O, O, B-PER, I-PER, O, O, O, O, B-MISC, O, O, O, O, O]"
8,10781,"[Montpellier, 4, 0, 2, 2, 3, 5, 2]","[NNP, CD, CD, CD, CD, CD, CD, CD]","[B-NP, I-NP, I-NP, I-NP, I-NP, I-NP, I-NP, I-NP]","[B-ORG, O, O, O, O, O, O, O]"
9,10072,"[8., Giuseppe, Citterio, (, Italy, ), Aki]","[CD, NNP, NNP, (, NNP, ), NNP]","[B-NP, I-NP, I-NP, O, B-NP, O, B-NP]","[O, B-PER, I-PER, O, B-LOC, O, B-PER]"


In [16]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [17]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [18]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 7592, 1010, 2023, 2003, 2028, 6251, 999, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [19]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [101, 7592, 1010, 2023, 2003, 2028, 6251, 3975, 2046, 2616, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [20]:
example = datasets["train"][4]
print(example["tokens"])

['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']


In [21]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'germany', "'", 's', 'representative', 'to', 'the', 'european', 'union', "'", 's', 'veterinary', 'committee', 'werner', 'z', '##wing', '##mann', 'said', 'on', 'wednesday', 'consumers', 'should', 'buy', 'sheep', '##me', '##at', 'from', 'countries', 'other', 'than', 'britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.', '[SEP]']


In [22]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(31, 39)

In [23]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 11, 11, 12, 13, 14, 15, 16, 17, 18, 18, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, None]


In [24]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

39 39


In [25]:
label_all_tokens = True

In [26]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [27]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102], [101, 2848, 13934, 102], [101, 9371, 2727, 1011, 5511, 1011, 2570, 102], [101, 1996, 2647, 3222, 2056, 2006, 9432, 2009, 18335, 2007, 2446, 6040, 2000, 10390, 2000, 18454, 2078, 2329, 12559, 2127, 6529, 5646, 3251, 5506, 11190, 4295, 2064, 2022, 11860, 2000, 8351, 1012, 102], [101, 2762, 1005, 1055, 4387, 2000, 1996, 2647, 2586, 1005, 1055, 15651, 2837, 14121, 1062, 9328, 5804, 2056, 2006, 9317, 10390, 2323, 4965, 8351, 4168, 4017, 2013, 3032, 2060, 2084, 3725, 2127, 1996, 4045, 6040, 2001, 24509, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100], [-100, 1, 2, -100], [-100, 5, 0, 

In [28]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6/cache-79c828cb7f0a8135.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6/cache-8967a832014ae656.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6/cache-7841b20b8dfe433c.arrow


In [29]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN t

In [30]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    output_dir='./Models/'
    f"{model_name}-finetuned-{task}-{dataset_name}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

In [31]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [32]:
metric = load_metric("seqeval")

In [33]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'LOC': {'f1': 1.0, 'number': 2, 'precision': 1.0, 'recall': 1.0},
 'ORG': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'PER': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'overall_accuracy': 1.0,
 'overall_f1': 1.0,
 'overall_precision': 1.0,
 'overall_recall': 1.0}

In [34]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [35]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [36]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: ner_tags, pos_tags, tokens, id, chunk_tags.
***** Running training *****
  Num examples = 14041
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2634


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.232800,0.071014,0.910592,0.919454,0.915001,0.980714
2,0.051600,0.063961,0.920724,0.932878,0.926762,0.983097
3,0.030900,0.062466,0.924088,0.935563,0.929790,0.983844


Saving model checkpoint to ./Models/distilbert-base-uncased-finetuned-ner-conll2003/checkpoint-500
Configuration saved in ./Models/distilbert-base-uncased-finetuned-ner-conll2003/checkpoint-500/config.json
Model weights saved in ./Models/distilbert-base-uncased-finetuned-ner-conll2003/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./Models/distilbert-base-uncased-finetuned-ner-conll2003/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./Models/distilbert-base-uncased-finetuned-ner-conll2003/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: ner_tags, pos_tags, tokens, id, chunk_tags.
***** Running Evaluation *****
  Num examples = 3250
  Batch size = 16
Saving model checkpoint to ./Models/distilbert-base-uncased-finetuned-ner-conll2003/checkpoint-1000
Configuration saved in ./Models/distilbert-base-uncased-finetuned-

TrainOutput(global_step=2634, training_loss=0.08462045721330694, metrics={'train_runtime': 519.8946, 'train_samples_per_second': 81.022, 'train_steps_per_second': 5.066, 'total_flos': 509926772226690.0, 'train_loss': 0.08462045721330694, 'epoch': 3.0})

In [40]:
tokenizer.save_pretrained(f"./Models/{model_name}-finetuned-{task}-{dataset_name}")

tokenizer config file saved in ./Models/distilbert-base-uncased-finetuned-ner-conll2003/tokenizer_config.json
Special tokens file saved in ./Models/distilbert-base-uncased-finetuned-ner-conll2003/special_tokens_map.json


('./Models/distilbert-base-uncased-finetuned-ner-conll2003/tokenizer_config.json',
 './Models/distilbert-base-uncased-finetuned-ner-conll2003/special_tokens_map.json',
 './Models/distilbert-base-uncased-finetuned-ner-conll2003/vocab.txt',
 './Models/distilbert-base-uncased-finetuned-ner-conll2003/added_tokens.json',
 './Models/distilbert-base-uncased-finetuned-ner-conll2003/tokenizer.json')

# Load Model for Inference

Load model from local


In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification

In [13]:
model_name = model_checkpoint.split("/")[-1]

In [14]:
checkpoint = f"./Models/{model_name}-finetuned-{task}-{dataset_name}/checkpoint-2500"
local_tokenizer = AutoTokenizer.from_pretrained(checkpoint)
local_model = AutoModelForTokenClassification.from_pretrained(checkpoint)

In [15]:
sequences = ["my name is rory"]

In [16]:
processed_tokens = local_tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

In [17]:
processed_tokens

{'input_ids': tensor([[  101,  2026,  2171,  2003, 14285,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [19]:
output = local_model(**processed_tokens)

In [20]:
output

TokenClassifierOutput([('logits',
                        tensor([[[ 4.9423, -0.3229, -1.0807, -0.4125, -0.6576, -0.5974, -1.2248,
                                  -0.3076, -1.4462],
                                 [ 5.6123,  0.8192, -1.8911,  0.0407, -1.6959, -1.0449, -2.3904,
                                   0.1969, -2.0965],
                                 [ 6.2818, -0.0345, -1.9247, -1.1873, -0.9254, -1.5457, -2.2918,
                                   0.0107, -1.3770],
                                 [ 6.8595,  0.3056, -1.8906, -0.7639, -1.3946, -1.3316, -2.4464,
                                  -0.0879, -1.8452],
                                 [ 0.0722,  5.4515, -1.6258,  0.1709, -1.8075, -0.3845, -2.1102,
                                   0.2809, -2.1586],
                                 [ 1.7756, -0.1714, -1.0379, -1.0331, -0.8038,  1.3738, -0.9824,
                                   0.3795, -0.4671]]], grad_fn=<AddBackward0>))])

In [21]:
print(output.logits)

tensor([[[ 4.9423, -0.3229, -1.0807, -0.4125, -0.6576, -0.5974, -1.2248,
          -0.3076, -1.4462],
         [ 5.6123,  0.8192, -1.8911,  0.0407, -1.6959, -1.0449, -2.3904,
           0.1969, -2.0965],
         [ 6.2818, -0.0345, -1.9247, -1.1873, -0.9254, -1.5457, -2.2918,
           0.0107, -1.3770],
         [ 6.8595,  0.3056, -1.8906, -0.7639, -1.3946, -1.3316, -2.4464,
          -0.0879, -1.8452],
         [ 0.0722,  5.4515, -1.6258,  0.1709, -1.8075, -0.3845, -2.1102,
           0.2809, -2.1586],
         [ 1.7756, -0.1714, -1.0379, -1.0331, -0.8038,  1.3738, -0.9824,
           0.3795, -0.4671]]], grad_fn=<AddBackward0>)


In [22]:
from transformers import pipeline

# Replace this with your own checkpoint
token_classifier = pipeline(
    "token-classification", model=checkpoint, aggregation_strategy="simple"
)

In [23]:
string = "The show is on the Disney Channel. It airs at 8pm. It will be shown in spanish and english."
print(string)
print(len(string))

The show is on the Disney Channel. It airs at 8pm. It will be shown in spanish and english.
91


In [24]:
result = token_classifier("The show is on the Disney Channel. It airs at 8pm. It will be shown in spanish and english.")

In [25]:
result

[{'end': 18,
  'entity_group': 'LABEL_0',
  'score': 0.9992917,
  'start': 0,
  'word': 'the show is on the'},
 {'end': 25,
  'entity_group': 'LABEL_3',
  'score': 0.9389609,
  'start': 19,
  'word': 'disney'},
 {'end': 33,
  'entity_group': 'LABEL_4',
  'score': 0.8383672,
  'start': 26,
  'word': 'channel'},
 {'end': 70,
  'entity_group': 'LABEL_0',
  'score': 0.99962467,
  'start': 33,
  'word': '. it airs at 8pm. it will be shown in'},
 {'end': 78,
  'entity_group': 'LABEL_7',
  'score': 0.9600869,
  'start': 71,
  'word': 'spanish'},
 {'end': 82,
  'entity_group': 'LABEL_0',
  'score': 0.9991244,
  'start': 79,
  'word': 'and'},
 {'end': 90,
  'entity_group': 'LABEL_7',
  'score': 0.96025616,
  'start': 83,
  'word': 'english'},
 {'end': 91,
  'entity_group': 'LABEL_0',
  'score': 0.9996352,
  'start': 90,
  'word': '.'}]

In [26]:
label_list

NameError: ignored

## Visualise Entites

In [57]:
import spacy
from spacy import displacy

text = "Hi my name is Rory Hurley. I work for the Cabinet Office. I speak english and a little bit of spanish"

nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
displacy.render(doc, style='ent', jupyter=True, options={'distance': 90})

In [58]:
text = "My name is John Smith and I live in Paris"
entities = [
    ("Employee", 11, 21),  # John Smith
    ("Location", 36, 41),  # Paris
]

In [77]:
import spacy

def display_entities(text, entities):
  nlp = spacy.blank("en")
  doc = nlp(text)
  ents = []
  for ee in entities:
      ents.append(doc.char_span(ee[1], ee[2], ee[0]))
  doc.ents = ents
  displacy.render(doc, style='ent', jupyter=True, options={'distance': 90})

def tokenise_and_display(text):
  result = token_classifier(text)
  print(result)
  res_ents = [(i['entity_group'], i['start'], i['end']) for i in result]
  print(res_ents)
  display_entities(text, entities=res_ents)

In [78]:
display_entities(text, entities)

In [79]:
result[:2]

[{'end': 18,
  'entity_group': 'LABEL_0',
  'score': 0.9992917,
  'start': 0,
  'word': 'the show is on the'},
 {'end': 25,
  'entity_group': 'LABEL_3',
  'score': 0.9389609,
  'start': 19,
  'word': 'disney'}]

In [80]:
res_ents = [(i['entity_group'], i['start'], i['end']) for i in result]
res_ents

[('LABEL_0', 0, 18),
 ('LABEL_3', 19, 25),
 ('LABEL_4', 26, 33),
 ('LABEL_0', 33, 70),
 ('LABEL_7', 71, 78),
 ('LABEL_0', 79, 82),
 ('LABEL_7', 83, 90),
 ('LABEL_0', 90, 91)]

In [81]:
display_entities(string, res_ents)

In [83]:
tokenise_and_display('Hi my name is Rory Hurley. I work for the Google. I speak english and a little bit of spanish')

[{'entity_group': 'LABEL_0', 'score': 0.99691457, 'word': 'hi my name is', 'start': 0, 'end': 13}, {'entity_group': 'LABEL_1', 'score': 0.99715924, 'word': 'rory', 'start': 14, 'end': 18}, {'entity_group': 'LABEL_2', 'score': 0.9967144, 'word': 'hurley', 'start': 19, 'end': 25}, {'entity_group': 'LABEL_0', 'score': 0.99147236, 'word': '. i work for the', 'start': 25, 'end': 41}, {'entity_group': 'LABEL_3', 'score': 0.99024755, 'word': 'google', 'start': 42, 'end': 48}, {'entity_group': 'LABEL_0', 'score': 0.99923176, 'word': '. i speak', 'start': 48, 'end': 57}, {'entity_group': 'LABEL_7', 'score': 0.9831236, 'word': 'english', 'start': 58, 'end': 65}, {'entity_group': 'LABEL_0', 'score': 0.9989486, 'word': 'and a little bit of', 'start': 66, 'end': 85}, {'entity_group': 'LABEL_7', 'score': 0.96781296, 'word': 'spanish', 'start': 86, 'end': 93}]
[('LABEL_0', 0, 13), ('LABEL_1', 14, 18), ('LABEL_2', 19, 25), ('LABEL_0', 25, 41), ('LABEL_3', 42, 48), ('LABEL_0', 48, 57), ('LABEL_7', 58, 